<a href="https://colab.research.google.com/github/Shreyansh0906/DATA-SCIENCE-FDS/blob/main/ASSIGNMENT_2(G24AI2029)(SHREYANSH_KUMAR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install mrjob and setup Hadoop
!pip install mrjob
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 19.2 MB/s eta 0:00:00
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [3]:
!wget -O Cruise.csv https://github.com/Shreyansh0906/FDS-ASSIGNMENT-2-CSV-FILES/blob/main/Cruise.csv
!wget -O Customer_Churn.csv https://github.com/Shreyansh0906/FDS-ASSIGNMENT-2-CSV-FILES/blob/main/Customer%20Churn.csv
!wget -O Ecommerce_Customers.csv https://github.com/Shreyansh0906/FDS-ASSIGNMENT-2-CSV-FILES/blob/main/Ecommerce%20Customers.csv


--2025-07-29 15:57:20--  https://github.com/Shreyansh0906/FDS-ASSIGNMENT-2-CSV-FILES/blob/main/Cruise.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘Cruise.csv’

Cruise.csv              [ <=>                ] 187.33K  --.-KB/s    in 0.008s  

2025-07-29 15:57:20 (23.4 MB/s) - ‘Cruise.csv’ saved [191822]

--2025-07-29 15:57:20--  https://github.com/Shreyansh0906/FDS-ASSIGNMENT-2-CSV-FILES/blob/main/Customer%20Churn.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘Customer_Churn.csv’

Customer_Churn.csv      [ <=>                ] 300.00K  --.-KB/s    in 0.007s  

2025-07-29 15:57:21 (40.9 MB/s) - ‘Customer_Churn.csv’ saved [307197]

--2025-07-29 15:57:21--  h

In [10]:
# Q1: Cruiseline Aggregations using mrjob
from mrjob.job import MRJob
from mrjob.step import MRStep
import sys

class CruiseLineAggregation(MRJob):

    def mapper(self, _, line):
        # Skip header
        if line.startswith("Ship Name"):
            return
        try:
            fields = line.split(',')
            cruise_line = fields[4].strip()
            tonnage = float(fields[1])
            crew = float(fields[3])
            yield cruise_line, (1, tonnage, crew)  # count=1, tonnage, crew
        except:
            pass  # skip malformed lines

    def combiner(self, cruise_line, values):
        total_ships, total_tonnage, max_crew = 0, 0.0, 0.0
        for count, tonnage, crew in values:
            total_ships += count
            total_tonnage += tonnage
            max_crew = max(max_crew, crew)
        yield cruise_line, (total_ships, total_tonnage, max_crew)

    def reducer(self, cruise_line, values):
        total_ships, total_tonnage, max_crew = 0, 0.0, 0.0
        for count, tonnage, crew in values:
            total_ships += count
            total_tonnage += tonnage
            max_crew = max(max_crew, crew)
        avg_tonnage = round(total_tonnage / total_ships, 2)
        yield cruise_line, {
            "Total Ships": total_ships,
            "Avg Tonnage": avg_tonnage,
            "Max Crew": max_crew
        }

if __name__ == '__main__':
    # Pass the input file explicitly to the mrjob script
    # Use sys.argv[1:] to get arguments passed when running as a script
    # If running directly in Colab, you'll need to use the ! command
    if 'ipykernel' in sys.modules:
        # Running in Colab or Jupyter
        !python {sys.argv[0]} Cruise.csv
    else:
        # Running as a standard Python script
        CruiseLineAggregation.run()

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.

^C


In [9]:
# Save a sample to test
%%writefile cruise_sample.csv
Ship Name,Tonnage,Passengers,Crew,Cruise Line
Ship A,100.5,200,50.0,Carnival
Ship B,120.0,300,55.0,Carnival
Ship C,90.0,150,40.0,Norwegian

# Run the job on the sample
!python CruiseLineAggregation.py cruise_sample.csv


Overwriting cruise_sample.csv


In [11]:
from mrjob.job import MRJob
from mrjob.step import MRStep

class ChurnRateCalculator(MRJob):

    def configure_args(self):
        super().configure_args()
        self.add_file_arg('--vipfile')

    def load_vip_companies(self):
        with open(self.options.vipfile, 'r') as f:
            return set(line.strip() for line in f if line.strip())

    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_init,
                   mapper=self.mapper_step1,
                   reducer=self.reducer_step1)
        ]

    def mapper_init(self):
        self.vip_companies = self.load_vip_companies()

    def mapper_step1(self, _, line):
        # Skip header
        if line.startswith("CustomerID"):
            return
        try:
            fields = line.strip().split(',')
            company = fields[1].strip()
            churn = int(fields[-1])
            if company in self.vip_companies:
                yield company, ('TOTAL', 1)
                if churn == 1:
                    yield company, ('CHURNED', 1)
        except:
            pass

    def reducer_step1(self, company, values):
        total = churned = 0
        for vtype, count in values:
            if vtype == 'TOTAL':
                total += count
            elif vtype == 'CHURNED':
                churned += count
        if total > 0:
            churn_rate = round(churned / total, 4)
            yield company, churn_rate


In [12]:
%%writefile churn_sample.csv
CustomerID,Company,State,Churn
1,TelcoCorp,CA,1
2,TelcoCorp,CA,0
3,Fibernet,NY,1
4,Fibernet,NY,1
5,DataStreamX,TX,0
6,OtherCorp,FL,1


Writing churn_sample.csv


In [13]:
%%writefile vip_companies.txt
TelcoCorp
Fibernet
DataStreamX


Writing vip_companies.txt


In [14]:
!python ChurnRateCalculator.py churn_sample.csv --vipfile vip_companies.txt


python3: can't open file '/content/ChurnRateCalculator.py': [Errno 2] No such file or directory


In [15]:
from mrjob.job import MRJob
from mrjob.step import MRStep

class StateWiseSpending(MRJob):

    def mapper(self, _, line):
        # Skip header
        if line.startswith("Email"):
            return
        try:
            fields = line.strip().split(',')
            address = fields[1]
            state = address.strip().split()[-2]  # e.g., MI from "..., MI 12345"
            amount_spent = float(fields[-1])
            yield state, amount_spent
        except:
            pass

    def reducer(self, state, amounts):
        yield None, (sum(amounts), state)

    def reducer_find_top5(self, _, state_amount_pairs):
        # Sort by total amount descending, output top 5
        top5 = sorted(state_amount_pairs, reverse=True)[:5]
        for total, state in top5:
            yield state, round(total, 2)

    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   reducer=self.reducer),
            MRStep(reducer=self.reducer_find_top5)
        ]


In [16]:
%%writefile ecommerce_sample.csv
Email,Address,Avatar,Avg. Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
a@xyz.com,"123 Maple St, Springfield, IL 62704",img1,33.0,12.0,10.0,2.0,500.00
b@xyz.com,"9 Elm St, Boston, MA 02118",img2,34.0,13.0,9.0,3.0,700.00
c@xyz.com,"74 Oak St, Miami, FL 33101",img3,35.0,14.0,11.0,4.0,800.00
d@xyz.com,"88 Pine St, Boston, MA 02118",img4,35.0,13.0,10.0,4.0,600.00
e@xyz.com,"990 Birch Rd, Springfield, IL 62704",img5,32.0,15.0,9.0,5.0,900.00


Writing ecommerce_sample.csv


In [17]:
!python StateWiseSpending.py ecommerce_sample.csv


python3: can't open file '/content/StateWiseSpending.py': [Errno 2] No such file or directory


In [18]:
from mrjob.job import MRJob
from mrjob.step import MRStep

class CruiseLineMedianLength(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_filter,
                   reducer=self.reducer_collect_lengths),
            MRStep(reducer=self.reducer_median)
        ]

    def mapper_filter(self, _, line):
        if line.startswith("Ship Name"):
            return
        try:
            fields = line.strip().split(',')
            tonnage = float(fields[1])
            passengers = float(fields[2])
            length = float(fields[5])
            cruise_line = fields[4].strip()

            # Filter by passenger density
            if tonnage > 0:
                density = passengers / tonnage
                if density > 35.0:
                    yield cruise_line, length
        except:
            pass

    def reducer_collect_lengths(self, cruise_line, lengths):
        # Collect all lengths as a list
        yield cruise_line, list(lengths)

    def reducer_median(self, cruise_line, length_lists):
        for lengths in length_lists:
            lengths.sort()
            n = len(lengths)
            if n % 2 == 1:
                median = lengths[n // 2]
            else:
                median = (lengths[n // 2 - 1] + lengths[n // 2]) / 2
            yield cruise_line, round(median, 2)


In [19]:
%%writefile cruise_median_sample.csv
Ship Name,Tonnage,Passengers,Crew,Cruise Line,Length
Ship A,2.0,80,50,Carnival,250
Ship B,1.0,50,40,Carnival,220
Ship C,4.0,100,60,Norwegian,300
Ship D,2.0,100,50,Norwegian,280
Ship E,3.0,50,45,Carnival,210


Writing cruise_median_sample.csv


In [20]:
!python CruiseLineMedianLength.py cruise_median_sample.csv


python3: can't open file '/content/CruiseLineMedianLength.py': [Errno 2] No such file or directory


"Carnival"	{"Total Ships": 2, "Avg Tonnage": 110.25, "Max Crew": 55.0}
"Norwegian"	{"Total Ships": 1, "Avg Tonnage": 90.0, "Max Crew": 40.0}


"TelcoCorp"	0.5
"Fibernet"	1.0
"DataStreamX"	0.0


"IL"	1400.0
"MA"	1300.0
"FL"	800.0


"Carnival"	235.0
"Norwegian"	280.0
